In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if 'jpg' in filename:           
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
data_list = os.listdir("/kaggle/input/cat-and-dog/training_set/training_set/")
print(data_list)

In [3]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions, VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import tensorflow as tf

In [4]:
train_datasets = "/kaggle/input/cat-and-dog/training_set/training_set/"
test_datasets = "/kaggle/input/cat-and-dog/test_set/test_set/"

In [5]:
image_size = (150,150)
number_class = len(data_list)
batch_size = 10
number_epochs = 10
learningRate = 0.001

In [6]:
train_dataset_generator = ImageDataGenerator(rescale = 1./255, #range = [0, 1]
                                            rotation_range = 45,
                                            featurewise_center = True,
                                            featurewise_std_normalization = True,
                                            height_shift_range=0.2,
                                            width_shift_range=0.2,
                                            shear_range = 0.2,
                                            zoom_range = 0.2,
                                            channel_shift_range = 20,
                                            horizontal_flip = True,
                                            vertical_flip = True,
                                            validation_split = 0.2, 
                                            fill_mode="constant")

In [7]:
train_batches = train_dataset_generator.flow_from_directory(train_datasets,
                                                           target_size= image_size,
                                                           shuffle=True,
                                                           batch_size=batch_size,
                                                           subset='training',
                                                           seed=42,
                                                           class_mode = "binary")

In [8]:
validation_batches = train_dataset_generator.flow_from_directory(train_datasets,
                                                           target_size= image_size,
                                                           shuffle=True,
                                                           batch_size=batch_size,
                                                           subset='validation',
                                                           seed=42,
                                                           class_mode = "binary")

In [9]:
vgg_model = VGG16(weights="imagenet",
                 include_top = False,
                 input_shape=(150, 150, 3))

vgg_model.trainable = False

In [10]:
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(4096, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [11]:
model.compile(loss = 'binary_crossentropy',
             optimizer = Adam(learning_rate = learningRate), 
             metrics = ['accuracy'])

In [12]:
model.summary()

In [13]:
print(len(train_batches))
print()
print(len(validation_batches))

In [14]:
print(train_batches.n)
print()
print(validation_batches.n)

In [15]:
print(train_batches.batch_size)
print()
print(validation_batches.batch_size)

In [16]:
train_step_size = train_batches.n // train_batches.batch_size
validation_step_size = validation_batches.n //validation_batches.batch_size

In [17]:
model_runing_history = model.fit(train_batches, 
                                steps_per_epoch = train_step_size,
                                validation_data=validation_batches,
                                validation_steps=validation_step_size,
                                epochs = number_epochs)